In [ ]:
from IPython.display import display, Markdown, HTML

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import missingno

In [ ]:
sns.set()
pd.options.mode.chained_assignment = None

# Lendo os dados
A base de dados utilizada está no [formato csv](https://www.linkedin.com/pulse/lendo-arquivos-csv-com-pandas-rog%C3%A9rio-guimar%C3%A3es-de-campos-j%C3%BAnior/)


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/nubank/diversidados-curso-ds/master/iniciante/base_imoveis.csv")

Olhando conteúdo das 5 primeiras linhas

In [ ]:
df.head()

Olhando nome das colunas

In [ ]:
df.columns

Olhando informações gerais sobre a base

In [ ]:
df.info()

Plotando matriz de nulos

In [ ]:
missingno.matrix(df)

Acima, vimos que não temos nenhum nulo na base! **Mas será que temos na base outros valores que poderiam ser considerados como nulo?**

Imprimindo tipo de cada coluna

In [ ]:
df.dtypes

In [ ]:
target = "preco_dolares"

# Explorando as variáveis
<img src="imgs/tipos_variaveis.png" alt="tipos variaveis" width="100%"/>

Para printar numero de linhas por cada valor possível da coluna `tem_porao`: `df["tem_porao"].value_counts()`

In [ ]:
df["tem_porao"].value_counts()

## Categoricas

### Cardinalidade

In [ ]:
cardinalidades = [] # lista que vai guardar valor das cardinalidades
vars_categoricas_raw = [] # lista que vai guardar nomes das variaveis categoricas
for i, row in df.dtypes.reset_index().iterrows():
    nome_var = row["index"]
    tipo_var = row[0]
    if tipo_var==object:
        cardinalidade = df[nome_var].nunique() # Funcao nunique retorna número de valores diferentes na coluna
        cardinalidades.append(cardinalidade) # Adc valor na lista
        vars_categoricas_raw.append(nome_var) # Adc nome na lista    
df_cardinalidades = pd.DataFrame({"Variavel": vars_categoricas_raw,
                                  "Cardinalidade": cardinalidades,}) # Criando uma tabela com valores calculados

In [ ]:
df_cardinalidades

Plotando cardinalidades

In [ ]:
sns.barplot(x="Cardinalidade", y="Variavel", data=df_cardinalidades, orient="h", palette="coolwarm")
plt.title("Cardinalidade das variaveis categoricas");

Colocando na escala logaritmica

In [ ]:
g = sns.barplot(x="Cardinalidade", y="Variavel", data=df_cardinalidades, orient="h", palette="coolwarm")
g.set_xscale("log")
plt.title("Cardinalidade das variaveis categoricas");

### Rank e distribuição

In [ ]:
for nome_var in vars_categoricas_raw:
    # Contando numero de casos em cada categoria e ordenando pelas classes mais frequentes 
    df_rank = df.groupby(nome_var).agg({"id": "count"}) \
                .sort_values("id", ascending=False)

    # Calculando porcentagem em cada categoria para ver distribuição
    df_rank["% ids"] = df_rank["id"]/df_rank["id"].sum()*100
    display(df_rank)

### Filtrando variáveis

In [ ]:
vars_categoricas_raw

In [ ]:
vars_categoricas_raw.pop(0)

In [ ]:
vars_categoricas_raw

In [ ]:
categoricas_nominais = ['orla_mar', 'tem_porao', 'teve_reforma']
categoricas_ordinais = [ 'nota_vista', 'nota_condicao',]

## Numericas

O método `.describe()` já calcula algumas métricas comuns nas variáveis numéricas

In [ ]:
df.describe()

Plot de distribuição tradicional

In [ ]:
vars_numericas_raw = []
for i, row in df.dtypes.reset_index().iterrows():
    nome_var = row["index"]
    tipo_var = row[0]
    if tipo_var in (np.float64, np.int64):
        print(nome_var)
        vars_numericas_raw.append(nome_var)
        fig, ax = plt.subplots()
        fig.set_size_inches(10, 6)
        sns.distplot(df[nome_var], hist=True, kde=False, norm_hist=False, ax=ax)
        plt.show()

In [ ]:
for nome_var in vars_numericas_raw:
    print(nome_var)
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 6)
    sns.boxplot(df[nome_var], ax=ax)
    plt.show()

### Filtrando variáveis

In [ ]:
vars_numericas_raw

In [ ]:
vars_numericas_raw.pop(0)

In [ ]:
vars_numericas_raw.pop(0)

In [ ]:
vars_numericas_raw

# Separando a base
<img src="imgs/splits.png" alt="separacao da base" width="80%"/>

Não existe um número mágico, alguns indicam 60%-20%-20%, e vamos usar essa proporção 

In [ ]:
vars_categoricas_raw

In [ ]:
vars_numericas_raw

Transformando campo de data pois estava como string

In [ ]:
df["data"] = pd.to_datetime(df["data"], format="%Y-%m-%d")

Contando numero de imoveis por dia

In [ ]:
df_datas = df.groupby("data").agg({"id": "count"})
df_datas["media_data"] = df_datas.rolling(7).mean()
df_datas = df_datas.reset_index()

Plotando número de imoveis por dia ao longo do tempo

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 6)
ax.plot(df_datas["data"], df_datas["media_data"])
plt.title("Numero de imoveis por data");

In [ ]:
df = df.sort_values("data").reset_index(drop="index")

## Divisão no tempo
Usando [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) da biblioteca sklearn.
Primeiro, fazemos a divisão no tempo, deixando 20% para nossa base de validação

In [ ]:
treino_teste, validacao = train_test_split(df, test_size=.2, random_state=0, shuffle=False)

Verificando se a divisão no tempo funcionou

In [ ]:
treino_teste.data.describe()

In [ ]:
validacao.data.describe()

## Divisão no espaço
Agora, dos 80% iniciais, vamos deixar 25% para teste e 75% para treino, misturando os dados para fazer apenas a divisão no espaço

In [ ]:
treino, teste = train_test_split(treino_teste, test_size=.25, random_state=0, shuffle=True)

In [ ]:
treino.data.describe()

In [ ]:
teste.data.describe()

## Check de tamanho das bases

In [ ]:
treino.shape, teste.shape, validacao.shape

# Tratando variáveis categóricas

## Ordinais

In [ ]:
categoricas_ordinais

Olhando categorias existentes

In [ ]:
treino["nota_vista"].unique()

Coloque os valores que representam a ordem das categorias

In [ ]:
nota_vista_map = {'Vista completamente bloqueada': 1, 
                  'Vista parcialmente bloqueada': 2, 
                  'Vista quase sem bloqueios': 3,
                  'Vista boa': 4,
                  'Excelente - vista sem bloqueios': 5}

In [ ]:
treino["nota_vista_encoded"] = treino["nota_vista"].map(nota_vista_map)
teste["nota_vista_encoded"] = teste["nota_vista"].map(nota_vista_map)
validacao["nota_vista_encoded"] = validacao["nota_vista"].map(nota_vista_map)

Faça o mesmo para variável `nota_condicao`

In [ ]:
treino["nota_condicao"].unique()

In [ ]:
nota_condicao_map = {'Excelente estado': 5,
                     'Bom estado' : 4, 
                     'Pronto para morar' : 3,
                     'Mau estado - pequena reforma necessária' : 2,
                     'Mau estado - reforma completa necessária' : 1}

In [ ]:
treino["nota_condicao_encoded"] = treino["nota_condicao"].map(nota_condicao_map)
teste["nota_condicao_encoded"] = teste["nota_condicao"].map(nota_condicao_map)
validacao["nota_condicao_encoded"] = validacao["nota_condicao"].map(nota_condicao_map)

## Nominais

In [ ]:
categoricas_nominais

In [ ]:
for var_n in categoricas_nominais:
    # Pegando média do target no treino para cada categoria
    var_map = treino.groupby(var_n).agg({target: "mean"}).to_dict()[target]
    
    treino[f"{var_n}_encoded"] = treino[f"{var_n}"].map(var_map) # Aplicando no treino
    teste[f"{var_n}_encoded"] = teste[f"{var_n}"].map(var_map) # Aplicando no teste
    validacao[f"{var_n}_encoded"] = validacao[f"{var_n}"].map(var_map) # Aplicando na validacao

In [ ]:
vars_categoricas = [x+"_encoded" for x in categoricas_nominais+categoricas_ordinais]

# Tratando variáveis numéricas

## Normalizando

In [ ]:
for var_n in vars_numericas_raw+vars_categoricas:
    scaler = MinMaxScaler()
    scaler.fit(treino[[var_n]]) # Usando treino como base para transformações
    treino[f"{var_n}_scaled"] = scaler.transform(treino[[var_n]]).reshape(-1) # Aplicando no treino
    teste[f"{var_n}_scaled"] = scaler.transform(teste[[var_n]]).reshape(-1) # Aplicando no teste
    validacao[f"{var_n}_scaled"] = scaler.transform(validacao[[var_n]]).reshape(-1) # Aplicando na validacao

# Organizando variáveis

Criamos várias colunas novas com as variáveis transformadas, precisamos filtrar só as novas, abaixo, temos todas as colunas que existem agora

In [ ]:
treino.columns

Agora vamos pegar só as que transformamos

In [ ]:
todas_variaveis = [x+"_scaled" for x in vars_numericas_raw+vars_categoricas]
todas_variaveis

Salvando quais eram as variaveis originais (raw)

In [ ]:
todas_variaveis_raw = vars_numericas_raw+vars_categoricas_raw

Olhando variáveis transformadas pra checar se estão ok

In [ ]:
validacao[todas_variaveis].head()

In [ ]:
validacao[todas_variaveis_raw].head()

# Modelinho teste

In [ ]:
todas_variaveis

In [ ]:
reg = DecisionTreeRegressor(random_state=0)
reg.fit(treino[vars_numericas_raw+vars_categoricas], treino[target])

In [ ]:
predictions_teste = reg.predict(teste[vars_numericas_raw+vars_categoricas])
predictions_validacao = reg.predict(validacao[vars_numericas_raw+vars_categoricas])

In [ ]:
r2_score(teste[target], predictions_teste), r2_score(validacao[target], predictions_validacao)

In [ ]:
r2_score(teste[target], predictions_teste), r2_score(validacao[target], predictions_validacao)